## Using nltk to analyze each instagram caption

In [1]:
import os
import nltk
from pathlib import Path
from pandas import Series, DataFrame
import pandas as pd
import matplotlib.pyplot as plt

p = Path(os.getcwd())
database_path=str(p.parent)+'/data/instagram.db'
print(database_path)

/Users/yutian/Desktop/final/data/instagram.db


In [2]:
import sqlite3

connection = sqlite3.connect(database_path)
instagram_data = pd.io.sql.read_sql("SELECT * FROM instagram;", con=connection)
instagram_data.head()

,id,account,filter,likes,comments,caption,timestamp
0,31044,murphyoilusa,Valencia,7,0,Elvis is in the building at Murphy Express 855...,140109 11:17
1,32057,murphyoilusa,Normal,5,0,Cool tanker! #murphyusa,140129 11:40
2,6359,murphyoilusa,Rise,6,0,Looking up! #murphyusa,121205 12:11
3,6347,murphyoilusa,Hefe,4,0,Quick Strike Energy! #murphyusa,121205 12:23
4,6351,murphyoilusa,X-Pro II,4,0,"Look out, it's Super Murph! #murphyusa",121205 12:16


In [3]:
instagram_data['focus'] = instagram_data.likes + instagram_data.comments
instagram_data['focus'].median()

215.0

In [4]:
instagram_data=instagram_data[['account','caption','timestamp','focus']]
instagram_data.sort_values(by='focus',ascending=0)[:10]

,account,caption,timestamp,focus
11330,nike,One giant step for man. #justdoit@nicolasmuell...,140514 13:39,263026
11349,nike,Teach the world what your country taught you. ...,140612 15:31,252340
11353,nike,"When your country rises, raise the volume. #ju...",140614 09:00,238977
11344,nike,Don�t let anything come between where you are ...,140605 17:30,232250
11196,nike,We can't live without Air.It's been 27 years s...,140325 21:00,217502
11213,nike,You did everything you could. Now do everythin...,140202 18:07,206829
11184,nike,An undersized QB shouldn't play so big. Runnin...,140203 00:24,197644
11346,nike,"When the dust settles, your legacy remains. #j...",140608 12:45,197357
11204,nike,Take the necessary steps. #justdoitOne month t...,140430 18:08,195816
11335,nike,You never know where the next step will take y...,140522 23:08,194088


In [8]:
# Top 10 most common words

caption_word=instagram_data.caption.apply(lambda x:x+"").sum()
caption_word=caption_word.lower()
print(type(caption_word))




<class 'str'>


In [9]:
from collections import Counter
Counter(caption_word.split()).most_common()[:10]

[('the', 20104),
 ('to', 11402),
 ('a', 9861),
 ('and', 9104),
 ('of', 8660),
 ('in', 8494),
 ('for', 7976),
 ('at', 6239),
 ('with', 5298),
 ('our', 5104)]

In [13]:
#Top 10 most common non-stopwords
stopwords = [str(word) for word in nltk.corpus.stopwords.words('english')]
title_words = [word for word in caption_word.split() if word.lower() not in stopwords]
Counter(title_words).most_common()[:20]

[('new', 1892),
 ('&', 1382),
 ('us', 1324),
 ('day', 1277),
 ('one', 1049),
 ('-', 1003),
 ('photo', 988),
 ('get', 975),
 ('like', 832),
 ('see', 821),
 ('today', 782),
 ('great', 711),
 ('love', 711),
 ('check', 651),
 ('look', 636),
 ('ready', 606),
 ('favorite', 602),
 ('thanks', 591),
 ('make', 585),
 ('stores', 581)]

In [15]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
bigram_finder = nltk.collocations.BigramCollocationFinder.from_words(title_words)

# Filter to only bigrams that appear 20+ times
bigram_finder.apply_freq_filter(20)

bigram_finder.score_ngrams(bigram_measures.raw_freq)[:20]

[(('ralph', 'lauren'), 0.0008344199649826469),
 (('chance', 'win'), 0.000707135563544616),
 (('stores', 'online'), 0.0006449076339526898),
 (("who's", 'copping?'), 0.0005232803170230158),
 (('foot', 'locker'), 0.0005091376057521235),
 (('show', 'us'), 0.00045822384517691116),
 (('#kohls', 'shop'), 0.00036771049304320033),
 (('thanks', 'sharing'), 0.00036488195078902185),
 (('#letsdothis', '#homedepot'), 0.00035073923951812954),
 (('air', 'jordan'), 0.0003394250705014157),
 (('take', 'pledge'), 0.00032528235923052335),
 (('new', 'york'), 0.0003224538169763449),
 (('lauren', 'collection'), 0.00028851130992620333),
 (('pledge', 'today'), 0.0002687115141469541),
 (('stores', 'online.'), 0.0002658829718927756),
 (('copping?', '#approved'), 0.00025456880287606174),
 (('locker', 'unlocked'), 0.00025456880287606174),
 (('photo', 'day'), 0.00025456880287606174),
 (('double', 'tap'), 0.0002517402606218833),
 (('tell', 'us'), 0.00024608317611352635)]

In [16]:
bigram_finder.nbest(bigram_measures.pmi, 10)

[('numbers', 'kohls.com:'),
 ('operation', 'lifesaver'),
 ('iheart', 'radio'),
 ('los', 'angeles'),
 ('item', 'numbers'),
 ('betty', 'crocker'),
 ('#styles', '#outfit'),
 ('#girl', '#girls'),
 ('#slopestyle', '#xgscene'),
 ('#tagsforlikes', '#me')]